In [1]:
import pandas as pd
import folium

In [2]:
file_url = 'http://www2.census.gov/geo/docs/maps-data/data/gazetteer/2016_Gazetteer/2016_Gaz_zcta_national.zip'

#Pandas usually infers zips are numerics, but we lose our leading zeroes so let's go with the object dtype
df = pd.read_csv(file_url, sep='\t', dtype={'GEOID' : object}) 

df.columns = df.columns.str.strip() #some column names have some padding
df.head()

,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,00601,166659884,799293,64.348,0.309,18.180555,-66.749961
1,00602,79287203,4448761,30.613,1.718,18.361945,-67.175597
2,00603,81884707,183895,31.616,0.071,18.455183,-67.119887
3,00606,109579998,12487,42.309,0.005,18.158345,-66.932911
4,00610,93020979,4171994,35.916,1.611,18.295366,-67.125135


In [3]:
#grab a random sample from df
subset_of_df = df.sample(n=100)

#creating a map that's centered to our sample
some_map = folium.Map(location=[subset_of_df['INTPTLAT'].mean(), 
                                subset_of_df['INTPTLONG'].mean()], 
                                zoom_start=4)

#creating a Marker for each point in df_sample.  Each point will get a popup with their zip
for row in subset_of_df.itertuples():
    some_map.add_child(folium.Marker(location=[row.INTPTLAT, row.INTPTLONG],
                                     popup=row.GEOID,
                                     icon=folium.Icon(color='green' if row.INTPTLONG > -100 else 'red', 
                                                      prefix='fa', icon='circle')))

legend_html =   '''
                <div style="position: fixed; 
                            bottom: 50px; left: 50px; width: 100px; height: 90px; 
                            border:2px solid grey; z-index:9999; font-size:14px;
                            ">&nbsp; Cool Legend <br>
                              &nbsp; East &nbsp; <i class="fa fa-map-marker fa-2x" style="color:green"></i><br>
                              &nbsp; West &nbsp; <i class="fa fa-map-marker fa-2x" style="color:red"></i>
                </div>
                ''' 

some_map.get_root().html.add_child(folium.Element(legend_html))
some_map